In [13]:
import os
from dotenv import load_dotenv
from azure.ai.ml import MLClient
from azure.identity import ClientSecretCredential

# Load environment variables
load_dotenv()

# Get Azure credentials from environment variables
subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID")
resource_group = os.getenv("AZURE_RESOURCE_GROUP")
workspace_name = os.getenv("AZURE_WORKSPACE_NAME")
tenant_id = os.getenv("AZURE_TENANT_ID")
client_id = os.getenv("AZURE_CLIENT_ID")
client_secret = os.getenv("AZURE_CLIENT_SECRET")

# Create credential and ML client
credential = ClientSecretCredential(
    tenant_id=tenant_id,
    client_id=client_id,
    client_secret=client_secret
)

ml_client = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name
)

# List all endpoints
endpoints = ml_client.online_endpoints.list()
for endpoint in endpoints:
    print(f"Endpoint Name: {endpoint.name}")
    print(f"  Status: {endpoint.provisioning_state}")
    print(f"  Scoring URI: {endpoint.scoring_uri}")
    print(f"  Authentication Mode: {endpoint.auth_mode}")
    print("-" * 50)

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed


Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


Endpoint Name: deberta-endpoint
  Status: Succeeded
  Scoring URI: http://194.171.191.227:30526/api/v1/endpoint/deberta-endpoint/score
  Authentication Mode: key
--------------------------------------------------


In [14]:
endpoint

KubernetesOnlineEndpoint({'provisioning_state': 'Succeeded', 'scoring_uri': 'http://194.171.191.227:30526/api/v1/endpoint/deberta-endpoint/score', 'openapi_uri': 'http://194.171.191.227:30526/api/v1/endpoint/deberta-endpoint/swagger.json', 'name': 'deberta-endpoint', 'description': None, 'tags': {}, 'properties': {'createdBy': 'a2230f31-0fda-428d-8c5c-ec79e91a49f5', 'createdAt': '2025-06-20T11:46:26.167926+0000', 'lastModifiedAt': '2025-06-20T12:33:44.357267+0000', 'azureml.onlineendpointid': '/subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/resourcegroups/buas-y2/providers/microsoft.machinelearningservices/workspaces/nlp6-2025/onlineendpoints/deberta-endpoint', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/providers/Microsoft.MachineLearningServices/locations/westeurope/mfeOperationsStatus/oeidp:1db9dd87-ebf5-4750-96c1-5679c7f557d6:74589479-ea40-402e-a188-03ebcaaaf628?api-version=2022-02-01-preview'}, 'print_as_yaml': Fal

In [ ]:
import requests
import json

# Get the endpoint keys for authentication
endpoint_keys = ml_client.online_endpoints.get_keys(name=endpoint.name)
primary_key = endpoint_keys.primary_key

# Prepare sample data for inference
sample_data = {
    "input_data": {
        "columns": ["text"],
        "index": [0],
        "data": [["I am so excited about this new project, it's going to be amazing!"]]
    }
}

# Set up headers with authentication
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json",
    "Authorization": f"Bearer {primary_key}"
}

# Make a POST request to the endpoint
response = requests.post(
    endpoint.scoring_uri,
    data=json.dumps(sample_data),
    headers=headers
)

# Print the response
print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")